In [1]:
import sys
sys.path.append('..')
from osp import *

df_meta = get_corpus_metadata()
worddb = get_worddb().fillna('')
ok_words = {word for word in worddb.index}

In [2]:
STASH_SLICES = HashStash('osp_slices_1000')
STASH_SLICES

Config,Param,Value
Path,Root Dir,/Users/ryan/.cache/hashstash/osp_slices_1000
,Filename,data.db
Engine,Engine,lmdb
,Serializer,hashstash
,Compress,lz4
,B64,True
Stats,Len,57620


In [3]:
tokenize("Hello, world!")

['hello', 'world']

In [4]:
def tokenize_agnostic(txt: str):
    """Tokenize text in a language-agnostic way.

    Args:
        txt: The input text.

    Returns:
        A list of tokens.
    """
    return re.findall(r"[\w']+|[.,!?; -—–'\n]", txt)

tokenize_agnostic("Hello, world!")

['Hello', ',', ' ', 'world', '!']

In [5]:
# def iter_corpus_txt_slices(slice_len=10):
#     num_recog = 0
#     slicex = []
#     for id, txt in iter_corpus_txt():
#         num_slices = 0
#         for si,sent in enumerate(txt.strip().split('\n')):
#             for word in tokenize_agnostic(sent):
#                 if word.strip() and word.lower() in ok_words:
#                     num_recog += 1
#                 slicex.append(word)
#             if num_recog >= slice_len:
#                 num_slices += 1
#                 otxt = ''.join(slicex).strip()
#                 yield id,num_slices,otxt
#                 slicex = []
#                 num_recog = 0
#             slicex.append('\n')



In [7]:
OK_WORDS = None
def get_ok_words():
    global OK_WORDS
    if OK_WORDS is None:
        worddb = get_worddb().fillna('')
        OK_WORDS = {word for word in worddb.index}
    return OK_WORDS


def iter_txt_slices(txt, slice_len=10, ok_words=None):
    num_recog = 0
    slicex = []
    num_slices = 0
    for si,sent in enumerate(txt.strip().split('\n')):
        for word in tokenize_agnostic(sent):
            if word.strip().isalpha() and word.lower() in ok_words:
                num_recog += 1
            slicex.append(word)
        if num_recog >= slice_len:
            num_slices += 1
            otxt = ''.join(slicex).strip()
            yield num_slices,otxt
            slicex = []
            num_recog = 0
        slicex.append('\n')


In [8]:
def get_text_slices(id, force=False, slice_len=1000):
    stash = HashStash(f'osp_slices_{slice_len}')
    if not force and id in stash:
        return stash[id]
    txt = get_corpus_txt(id)
    slices = dict(iter_txt_slices(txt, slice_len, get_ok_words()))
    stash[id] = slices
    return slices

In [9]:
# get_text_slices('phil/10.2307/40231690')

In [10]:
for id in tqdm(df_meta.index):
    get_text_slices(id)

100%|██████████| 57620/57620 [06:15<00:00, 153.37it/s]


In [128]:
stash

Config,Param,Value
Path,Root Dir,/Users/rj416/.cache/hashstash/osp_slices
,Dbname,xxx
,Filename,lmdb.hashstash.lz4+b64
Engine,Engine,lmdb
,Serializer,hashstash
,Compress,lz4
,B64,True


In [86]:
STASH_SLICES.path

'/Users/rj416/.cache/hashstash/osp_slices/lmdb.hashstash.lz4+b64/data.db'